# 0.6_ss_chap06_gradients
- gradient の実行結果を動かしてから、中身について見てみる

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [4]:
from src import utils
import itertools
import logging
import numpy as np
import matplotlib.pylab as plt
import matplotlib.patches as patches
%matplotlib inline

In [6]:
from src.common.optimizer import SGD, Momentum, Nesterov, AdaGrad
from src.common.optimizer import RMSprop, Adam

In [7]:
log_fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
logging.basicConfig(level=logging.INFO, format=log_fmt)
# logging.basicConfig(level=logging.DEBUG, format=log_fmt)
logger = logging.getLogger()

## from optimier_compare_naive